In [1]:
import pandas as pd

In [2]:
import openai

with open("../openai_api_key.txt", "r") as f:
    api_key = f.read()

# openai.api_key = api_key

GPT_model = "gpt-3.5-turbo"

In [3]:
RANDOM_SEED = 123

client = openai.Client(api_key=api_key)

In [4]:
# read descriptions from json file
from data_load import load_games, load_system_prompt

games = load_games()
system_prompt = load_system_prompt('benchmark_prompt.txt')

In [5]:
def get_model_response(message):
    completion = client.chat.completions.create(
    model=GPT_model,
    seed=RANDOM_SEED,
    messages=message,
    )
    return completion.choices[0].message.content

In [22]:
game = games[4]
path_name = game.code_file_path.split('/')[-1]
game_tag = path_name[:-4]
df = pd.read_csv(f'data/game_condition_query/{game_tag}.csv')
df = df[df.columns[1:]]
n_phrases_language = len(list(filter(lambda col_name: "language_phrase_" in col_name, df.columns)))
N_PHRASES = n_phrases_language

model_response = []
for _, row in df.iterrows():
    messages = []
    messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": game.description})
    for i in range(1, N_PHRASES + 1):
        if not pd.isna(row[f"language_phrase_{i}"]):
            messages.append({"role": "user", "content": row[f"language_phrase_{i}"]})
    messages.append({"role": "user", "content": row["question_sentence"]})
    model_response.append(get_model_response(messages))

output_df = df[['task_id', 'answer']].copy()
output_df['model_answer'] = model_response
output_df.to_csv(f'data/benchmark_output/{game_tag}.csv')

[{'role': 'system', 'content': "You are an assistant answering queries about games. The user will first give you the game description, then a list of moves that each player makes. The user will then ask you questions about the game. Your task is to answer the user's questions about the game given the game description and list of moves."}, {'role': 'user', 'content': "Two concentric squares with a line connecting their midpoints. Six pieces per player. Players alternate placing one of their pieces on the intersections of the lines. Whenever a player places three of their pieces in a row, they may remove one of the opponent's pieces from the board. Once all of the pieces are placed, players may take turns moving a piece to an adjacent point along the lines, attempting to place three of their pieces in a row. When one player is reduced to two pieces, that player loses."}, {'role': 'user', 'content': 'Player 1 places their piece on the upper right corner of the outer square.'}, {'role': 'u